In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.tensorflow

class MyModel:
    def __init__(self):
        # Load and prepare the California housing dataset
        housing = fetch_california_housing()
        X_train_full, X_test, y_train_full, y_test = train_test_split(
            housing.data, housing.target)
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train_full, y_train_full)

        # Scale the features
        self.scaler = StandardScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_valid = self.scaler.transform(X_valid)
        self.X_test = self.scaler.transform(X_test)
        self.y_test = y_test

        # Define the model architecture
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
            tf.keras.layers.Dense(1)
        ])

        # Compile the model
        self.model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

        # Fit the model
        self.history = self.model.fit(X_train, y_train, epochs=20,
                                      validation_data=(X_valid, y_valid))

    def predict(self, input_data):
        # Make predictions
        return self.model.predict(input_data)




2024-07-29 12:29:18.856626: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 12:29:18.863836: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 12:29:18.952261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 12:29:18.953989: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 12:29:19.839147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    my_model = MyModel()
        # Set the MLflow tracking URI to the local server

    # Log parameters
    mlflow.log_param("learning_rate", 1e-3)
    mlflow.log_param("epochs", 20)
    
    # Log metrics
    for epoch, loss in enumerate(my_model.history.history['loss']):
        mlflow.log_metric("loss", loss, step=epoch)
    for epoch, val_loss in enumerate(my_model.history.history['val_loss']):
        mlflow.log_metric("val_loss", val_loss, step=epoch)
    
    # Log the model
    mlflow.tensorflow.log_model(my_model.model, "model")
    
    # Perform inferencing with some sample inputs from the test set
    sample_inputs = my_model.X_test[:5]  # Take the first 5 samples from the test set
    predictions = my_model.predict(sample_inputs)
    
    # Print the predictions
    print("Sample inputs:\n", sample_inputs)
    print("Predictions:\n", predictions)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.2800 - val_loss: 0.8596
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7232 - val_loss: 0.6382
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6306 - val_loss: 0.5977
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6001 - val_loss: 0.5735
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5786 - val_loss: 0.5549
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5614 - val_loss: 0.5407
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5475 - val_loss: 0.5286
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5369 - val_loss: 0.5185
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5274 - val_loss: 0.5115
Epoch 10/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5197 - val_loss: 0.5038

2024/07/29 12:32:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp9ut6bvzo/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp9ut6bvzo/model/data/model/assets


1/1 [==============================] - 0s 68ms/step
Sample inputs:
 [[-0.33763231 -0.20627036 -0.42778554 -0.20885401 -0.75208509 -0.11795677
  -0.94377481  0.83431732]
 [-0.46735627  1.53609272  0.04164764 -0.04389985 -0.23715672 -0.01634571
  -0.77565481  0.61525903]
 [-0.49443001 -0.60226197 -0.0790481   0.12039601 -1.09327015 -0.0377849
   0.98493517 -0.62441176]
 [-0.79715409  0.26891957  0.14691952  0.08608743 -0.32357827 -0.02163727
  -0.46743482 -0.44518224]
 [-0.43003842 -0.44386532 -0.38140846 -0.25478021 -0.4316052  -0.01074768
  -0.72895481  0.95380366]]
Predictions:
 [[1.9226772]
 [2.0630178]
 [1.2439072]
 [2.2743993]
 [1.3383068]]


In [12]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load the model from the MLflow Model Registry
model_name = "MyModel"
model_version = 1  # Specify the version of the model you want to use
model = mlflow.tensorflow.load_model(f"models:/{model_name}/{model_version}")

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    predictions = model.predict(data['input'])
    return jsonify(predictions.tolist())



/home/jyothi/miniconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.22.158.244:5001
INFO:werkzeug:Press CTRL+C to quit
